# Weblog Exercise

We want to be able to perform analyses on the logs of a web server ("UofS_access_log.small"). To do this, you need to get the relevant data into a dataframe. This should be an automated process so that other log files can also be loaded.

The following tasks need to be done. The original dataframe should be reworked so that only these columns remain:

- domain: contains the addresses of the clients that sent a request
- timestamp: is a datetime field (POSIXct) that shows the time of the request
- resource: shows the resource that was requested
- response_code: gives the HTTP response code returned by the server
- response_length: indicates the length of the HTTP response


Import all necessary libraries here:

In [1]:
#SOLUTION_START
import pandas as pd
#SOLUTION_END

## Reading the data
Open the file "UofS_access_log.small" and investigate its contents. The file is a log file from a web server.

Read the dataframe.
- Check for yourself what the separator is.
- Incorrect rows can be skipped.
- There is no header!
- The file uses the "latin" encoding (consult the docs to learn how to set the encoding) for characters.

In [2]:
#SOLUTION_START
#log = pd.read_csv("../datasets/UofS_access_log.small.csv", sep=" ", encoding="latin", header=None, on_bad_lines='skip')

#This commented code below is the solution for the wrong type of response_length column.
log = pd.read_csv("../../datasets/UofS_access_log", sep=" ", encoding="latin", header=None, on_bad_lines='skip', na_values="-", dtype={2: 'string'})
log.head()
#SOLUTION_END

,0,1,2,3,4,5,6,7
0,202.32.92.47,NaN,<NA>,[01/Jun/1995:00:00:59,-0600],GET /~scottp/publish.html,200,271.0
1,ix-or7-27.ix.netcom.com,NaN,<NA>,[01/Jun/1995:00:02:51,-0600],GET /~ladd/ostriches.html,200,205908.0
2,ram0.huji.ac.il,NaN,<NA>,[01/Jun/1995:00:05:44,-0600],GET /~scottp/publish.html,200,271.0
3,eagle40.sasknet.sk.ca,NaN,<NA>,[01/Jun/1995:00:08:06,-0600],GET /~lowey/,200,1116.0
4,eagle40.sasknet.sk.ca,NaN,<NA>,[01/Jun/1995:00:08:19,-0600],GET /~lowey/kevin.gif,200,49649.0


Investigate the dataframe.

In [4]:
#SOLUTION_START
log.info()
log.describe()
#SOLUTION_END

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2408623 entries, 0 to 2408622
Data columns (total 8 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       object 
 1   1       float64
 2   2       string 
 3   3       object 
 4   4       object 
 5   5       object 
 6   6       int64  
 7   7       float64
dtypes: float64(2), int64(1), object(4), string(1)
memory usage: 147.0+ MB


,1,6,7
count,0.0,2.408623e+06,2.362336e+06
mean,NaN,2.102198e+02,5.479282e+03
std,NaN,3.400302e+01,6.396353e+04
min,NaN,2.000000e+02,0.000000e+00
25%,NaN,2.000000e+02,5.500000e+02
50%,NaN,2.000000e+02,1.723000e+03
75%,NaN,2.000000e+02,3.455000e+03
max,NaN,5.010000e+02,3.019382e+07


Delete columns 1 and 2 as they do not contain any relevant information. Use the ``drop()`` function on the dataframe.

In [3]:
#SOLUTION_START
log.drop(columns=[1,2], inplace=True)
#SOLUTION_END

Column 0 should be named "domain", column (with explicit index) 5 should be named "resource", 6 should be named "response_code", column 7 should be named response_length.Rename these columns in the dataframe by replacing log.columns with the correct names.


In [4]:
#SOLUTION_START
log.columns = ["domain",3,4, "resource", "response_code", "response_length"]
#SOLUTION_END

The "response_length" has the 'object' type but we expected it to be numerical. What is the reason? Try to solve the problem when reading the csv file.

In [58]:
#SOLUTION_START
# The CSV contains '-' in the response_length column.  This '-' probably indicates that the response length is unknown. We can replace these with NaN values.
# Add the na_values parameter to the read_csv function.
#SOLUTION_END

In the previous question you had to replace '-' with NaN values. How many NaN values are in response_length?

In [8]:
#SOLUTION_START
log["response_length"].isna().sum()
#SOLUTION_END

46287

What percentage is that of all rows?

In [60]:
#SOLUTION_START
log["response_length"].isna().sum() / len(log["response_length"]) * 100
#SOLUTION_END

1.9217204186790542

The timestamp is spread across columns 3 (date and time) and 4 (timezone). Combine these into one string. Place the result in a variable "timestamp" and drop the original columns.

In [61]:
#SOLUTION_START
log['timestamp']= log.loc[:,3] + log.loc[:,4]
log.drop(columns=[3,4], inplace=True)
#SOLUTION_END


Remove the 'GET' and 'HTTP/1.0' that sometimes appear at the beginning and end of the "resource" column.

In [62]:
#SOLUTION_START
log["resource"] = log["resource"].str.replace("GET ", "", regex=False).str.replace("HTTP/1.0", "", regex=False)
#SOLUTION_END

Remove all rows from your dataframe where a missing value occurs.

In [63]:
#SOLUTION_START
log.dropna(inplace=True)
log.head()
#SOLUTION_END

,domain,resource,response_code,response_length,timestamp
0,202.32.92.47,/~scottp/publish.html,200,271.0,[01/Jun/1995:00:00:59-0600]
1,ix-or7-27.ix.netcom.com,/~ladd/ostriches.html,200,205908.0,[01/Jun/1995:00:02:51-0600]
2,ram0.huji.ac.il,/~scottp/publish.html,200,271.0,[01/Jun/1995:00:05:44-0600]
3,eagle40.sasknet.sk.ca,/~lowey/,200,1116.0,[01/Jun/1995:00:08:06-0600]
4,eagle40.sasknet.sk.ca,/~lowey/kevin.gif,200,49649.0,[01/Jun/1995:00:08:19-0600]


Find the row(s) with the largest response_length.

In [64]:
#SOLUTION_START
rows = log.response_length == log.response_length.max()
log.loc[rows]
#SOLUTION_END

,domain,resource,response_code,response_length,timestamp
127004,mac40199.usask.ca,/uofs/ivany_movie.mov,200,30193824.0,[19/Jun/1995:16:32:20-0600]
127629,duke.usask.ca,/uofs/ivany_movie.mov,200,30193824.0,[19/Jun/1995:17:59:05-0600]
131615,agora.carleton.ca,/uofs/ivany_movie.mov,200,30193824.0,[20/Jun/1995:08:47:16-0600]
144938,krause.usask.ca,/uofs/ivany_movie.mov,200,30193824.0,[21/Jun/1995:13:16:53-0600]
161552,grapes.usask.ca,/uofs/ivany_movie.mov,200,30193824.0,[23/Jun/1995:11:09:40-0600]
164035,palona1.cns.hp.com,/uofs/ivany_movie.mov,200,30193824.0,[23/Jun/1995:15:23:04-0600]


Save the result in a CSV file "log_result.csv". Use ',' as the separator and "." for decimal numbers.

In [65]:
#SOLUTION_START
log.to_csv("log_result.csv", sep=",", decimal=".", index=False)
#SOLUTION_END

Now, it is time to check that the solution also works for the full dataset "UofS_access_log". Perform the same steps as above. Mind that the file is 227Mb and processing can take a while, but should not take ages.
When I ran the code with the full data set, it took about 1 minute.\
I also got a warning ``DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.`` Pandas can only know the dtypes after reading the entire file. This is why it is recommended to set the dtype explicitly when reading the file. I added ``dtype={2: 'string'}`` to the read_csv function.